In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip /content/drive/MyDrive/data/archive.zip

Archive:  /content/drive/MyDrive/data/archive.zip
  inflating: test_data/test_data/blasti/DSC_6396.jpg  
  inflating: test_data/test_data/blasti/DSC_6397.jpg  
  inflating: test_data/test_data/blasti/DSC_6398.jpg  
  inflating: test_data/test_data/blasti/DSC_6399.jpg  
  inflating: test_data/test_data/blasti/DSC_6400.jpg  
  inflating: test_data/test_data/blasti/DSC_6401.jpg  
  inflating: test_data/test_data/blasti/DSC_6402.jpg  
  inflating: test_data/test_data/blasti/DSC_6403.jpg  
  inflating: test_data/test_data/blasti/DSC_6405.jpg  
  inflating: test_data/test_data/blasti/DSC_6406.jpg  
  inflating: test_data/test_data/blasti/DSC_6407.jpg  
  inflating: test_data/test_data/blasti/DSC_6408.jpg  
  inflating: test_data/test_data/blasti/DSC_6409.jpg  
  inflating: test_data/test_data/blasti/DSC_6410.jpg  
  inflating: test_data/test_data/blasti/DSC_6411.jpg  
  inflating: test_data/test_data/bonegl/DSC_4587.jpg  
  inflating: test_data/test_data/bonegl/DSC_4588.jpg  
  inflating: te

In [ ]:
# Data Augmentation

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_gen = ImageDataGenerator(rescale=(1./255),horizontal_flip=True,shear_range=0.2)
test_gen = ImageDataGenerator(rescale=(1./255))

In [ ]:
train = train_gen.flow_from_directory('/content/train_data/train_data',target_size=(120, 120),class_mode='categorical', batch_size=8)
test = test_gen.flow_from_directory('/content/test_data/test_data',target_size=(120, 120),class_mode='categorical',batch_size=8)

Found 150 images belonging to 16 classes.
Found 157 images belonging to 16 classes.


In [ ]:
train.class_indices

{'blasti': 0,
 'bonegl': 1,
 'brhkyt': 2,
 'cbrtsh': 3,
 'cmnmyn': 4,
 'gretit': 5,
 'hilpig': 6,
 'himbul': 7,
 'himgri': 8,
 'hsparo': 9,
 'indvul': 10,
 'jglowl': 11,
 'lbicrw': 12,
 'mgprob': 13,
 'rebimg': 14,
 'wcrsrt': 15}

In [ ]:
# CNN

from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Dropout
from tensorflow.keras.optimizers.legacy import Adam

In [ ]:
model = Sequential()

model.add(Convolution2D(12,(3,3),activation='relu',input_shape=(120, 120, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Convolution2D(24,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Convolution2D(36,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(62,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='relu'))

model.add(Dense(16,activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_27 (Conv2D)          (None, 118, 118, 12)      336       
                                                                 
 batch_normalization_35 (Bat  (None, 118, 118, 12)     48        
 chNormalization)                                                
                                                                 
 max_pooling2d_27 (MaxPoolin  (None, 59, 59, 12)       0         
 g2D)                                                            
                                                                 
 dropout_35 (Dropout)        (None, 59, 59, 12)        0         
                                                                 
 conv2d_28 (Conv2D)          (None, 57, 57, 24)        2616      
                                                                 
 batch_normalization_36 (Bat  (None, 57, 57, 24)     

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
# Early stopping
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stop = EarlyStopping(monitor='accuracy',patience=8)

In [ ]:
model.fit(train,batch_size=8,validation_data=test,epochs=20,callbacks=early_stop)

Epoch 1/20
19/19 [==============================] - 113s 6s/step - loss: 2.5058 - accuracy: 0.1533 - val_loss: 2.7874 - val_accuracy: 0.1338
Epoch 2/20
19/19 [==============================] - 113s 6s/step - loss: 2.3555 - accuracy: 0.2600 - val_loss: 2.8277 - val_accuracy: 0.1274
Epoch 3/20
19/19 [==============================] - 114s 6s/step - loss: 2.1434 - accuracy: 0.3600 - val_loss: 2.8906 - val_accuracy: 0.1274
Epoch 4/20
19/19 [==============================] - 80s 4s/step - loss: 2.1241 - accuracy: 0.3400 - val_loss: 2.7966 - val_accuracy: 0.1465
Epoch 5/20
19/19 [==============================] - 77s 4s/step - loss: 1.9602 - accuracy: 0.4333 - val_loss: 2.8574 - val_accuracy: 0.0764
Epoch 6/20
19/19 [==============================] - 74s 4s/step - loss: 1.8476 - accuracy: 0.4067 - val_loss: 2.9059 - val_accuracy: 0.0764
Epoch 7/20
19/19 [==============================] - 75s 4s/step - loss: 1.7972 - accuracy: 0.4933 - val_loss: 3.1343 - val_accuracy: 0.0510
Epoch 8/20
19/19 

In [ ]:
model.save('birdClassifier.h5')

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

In [ ]:

img1 = image.load_img('/content/himgri.jpg',target_size=(120,120))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1,axis=0)
pred = np.argmax(model.predict(img1))
output=list(train.class_indices)
print(pred)
print(output[pred])

1/1 [==============================] - 0s 23ms/step
8
himgri
